<a href="https://colab.research.google.com/github/olalekanowolabi64/bug-free-octo-invention/blob/master/Classification_in_PySpark_Big_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification in PySpark
> Now that you are familiar with getting data into Spark, you'll move onto building two types of classification model - Decision Trees and Logistic Regression. 

credit: This notebook is inspired from datacamp machine learning with spark course

#Setting up PySpark in Colab
Spark is written in the Scala programming language and requires the Java Virtual Machine (JVM) to run. Therefore, our first task is to download Java.


In [ ]:
!apt-get install openjdk-8-jdk-headless

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 42 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jre-headless amd64 8u312-b07-0ubuntu1~18.04 [28.2 MB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jdk-headless amd64 8u312-b07-0ubuntu1~18.

Now install Spark 3.2.1 with Hadoop 2.7

In [ ]:
!wget https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz


--2022-05-19 08:31:43--  https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
Resolving archive.apache.org (archive.apache.org)... 138.201.131.134, 2a01:4f8:172:2ec5::2
Connecting to archive.apache.org (archive.apache.org)|138.201.131.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 272637746 (260M) [application/x-gzip]
Saving to: ‘spark-3.2.1-bin-hadoop2.7.tgz’

spark-3.2.1-bin-had 100%[===================>] 260.01M  24.8MB/s    in 11s     

2022-05-19 08:31:55 (23.8 MB/s) - ‘spark-3.2.1-bin-hadoop2.7.tgz’ saved [272637746/272637746]



 we just need to unzip that folder.


In [ ]:
!tar xf /content/spark-3.2.1-bin-hadoop2.7.tgz


There is one last thing that we need to install and that is the findspark library. It will locate Spark on the system and import it as a regular library.


In [ ]:
!pip install -q findspark


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

We need to locate Spark in the system. For that, we import findspark and use the findspark.init() method.

In [ ]:
import findspark
findspark.init()
findspark.find()

'/content/spark-3.2.1-bin-hadoop2.7'

Now that we have installed all the necessary dependencies in Colab, it is time to set the environment path. This will enable us to run Pyspark in the Colab environment.

In [ ]:
import pyspark
import numpy as np
import pandas as pd

#Problem description

In this exercise, you will develop two classifiers to perform a binary classification task - The objective is to predict whether a flight is likely to be delayed by at least 15 minutes (label 1) or not (label 0).

## Data Preparation

### Removing columns and rows
You previously loaded airline flight data from a CSV file. You're going to develop a model which will predict whether or not a given flight will be delayed.

In this exercise you need to trim those data down by:

1. removing an uninformative column and
2. removing rows which do not have information about whether or not a flight was delayed.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').appName('Catch_Pink_Flamingo').getOrCreate()

# Read data from CSV file
combined_data = spark.read.csv('/content/combined-data1.csv', sep=',', header=True, inferSchema=True, nullValue='NA')

combined_data.show(5)


+------+-------------+---------+------------+----------------+----------+-----------+---------+
|userId|userSessionId|teamLevel|platformType|count_gameclicks|count_hits|count_buyId|avg_price|
+------+-------------+---------+------------+----------------+----------+-----------+---------+
|   812|         5648|        1|     android|              69|         8|       null|     null|
|  1658|         5649|        1|      iphone|              31|         5|       null|     null|
|  1589|         5650|        1|      iphone|              26|         2|       null|     null|
|  1863|         5651|        1|     android|              35|         4|       null|     null|
|   937|         5652|        1|     android|              39|         0|          1|      1.0|
+------+-------------+---------+------------+----------------+----------+-----------+---------+
only showing top 5 rows



#Total number of records

In [ ]:
combined_data.count()

4619

In [ ]:
# Remove the 'userId' column
combined_data_drop_column = combined_data.drop('userId','userSessionId')
# Number of records with missing 'avg_price' values
combined_data_drop_column.filter('avg_price is NULL').count()

# Remove records with missing 'avg_price' values
valid_avg_price = combined_data_drop_column.filter('avg_price IS NOT NULL')

# Remove records with missing values in any column and get the number of remaining rows
combined_data_cleaned = valid_avg_price.dropna()
print(combined_data_cleaned.count())

1411


### Column manipulation
The Federal Aviation Administration (FAA) considers a flight to be "delayed" when it arrives 15 minutes or more after its scheduled time.

The next step of preparing the flight data has two parts:

1. convert the units of distance, replacing the `mile` column with a `kmcolumn`; and
2. create a Boolean column indicating whether or not a flight was delayed.

In [ ]:
from pyspark.sql.functions import round


# Create 'label' column indicating whether a user is HighRoller (1) or PennyPincher(0)
combined_data_cleaned = combined_data_cleaned.withColumn('label', (combined_data_cleaned.avg_price >5).cast('integer'))

# Check first five records
combined_data_cleaned.show(20)


+---------+------------+----------------+----------+-----------+---------+-----+
|teamLevel|platformType|count_gameclicks|count_hits|count_buyId|avg_price|label|
+---------+------------+----------------+----------+-----------+---------+-----+
|        1|     android|              39|         0|          1|      1.0|    0|
|        1|      iphone|             129|         9|          1|     10.0|    1|
|        1|     android|             102|        14|          1|      5.0|    0|
|        1|     android|              39|         4|          1|      3.0|    0|
|        1|     android|              90|        10|          1|      3.0|    0|
|        1|      iphone|              51|         8|          1|     20.0|    1|
|        1|     android|              51|         6|          2|      2.5|    0|
|        1|     android|              47|         5|          2|      2.0|    0|
|        1|     android|              46|         7|          1|      1.0|    0|
|        1|      iphone|    

### Categorical columns
In the flights data there are two columns, `carrier` and `org`, which hold categorical data. You need to transform those columns into indexed numerical values.



In [ ]:
from pyspark.ml.feature import StringIndexer

# Create an indexer
indexer = StringIndexer(inputCol='platformType', outputCol='platformType_idx')

# Indexer identifies categories in the data
indexer_model = indexer.fit(combined_data_cleaned)

# Indexer creates a new column with numeric index values
combData_indexed = indexer_model.transform(combined_data_cleaned)

# Repeat the process for the other categorical feature
#combData_indexed = StringIndexer(inputCol='org', outputCol='org_idx').fit(combData_indexed).transform(flights_indexed)

### Assembling columns
The final stage of data preparation is to consolidate all of the predictor columns into a single column.

At present our data has the following predictor columns:

- `mon`, `dom` and `dow`
- `carrier_idx` (derived from `carrier`)
- `org_idx` (derived from `org`)
- `km`
- `depart`
- `duration`

In [ ]:
from pyspark.ml.feature import VectorAssembler

# Create an assembler object
assembler = VectorAssembler(inputCols=[
    'teamLevel', 'platformType_idx', 'count_gameclicks',
    'count_hits', 
    'count_buyId',
], outputCol='features')

# Consolidate predictor columns
combData_assembled = assembler.transform(combData_indexed)

# Check the resulting column
combData_assembled.select('features', 'avg_price').show(5, truncate=False)

+------------------------+---------+
|features                |avg_price|
+------------------------+---------+
|[1.0,1.0,39.0,0.0,1.0]  |1.0      |
|[1.0,0.0,129.0,9.0,1.0] |10.0     |
|[1.0,1.0,102.0,14.0,1.0]|5.0      |
|[1.0,1.0,39.0,4.0,1.0]  |3.0      |
|[1.0,1.0,90.0,10.0,1.0] |3.0      |
+------------------------+---------+
only showing top 5 rows



## Decision Tree


### Train/test split
To objectively assess a Machine Learning model you need to be able to test it on an independent set of data. You can't use the same data that you used to train the model: of course the model will perform (relatively) well on those data!

You will split the data into two components:

- training data (used to train the model) and
- testing data (used to test the model).

In [ ]:
# Split into training and test sets in a 80:20 ratio
combData_train, combData_test = combData_assembled.randomSplit([0.8, 0.2], seed=17)

# Check that training set has around 80% of records
training_ratio = combData_train.count() / combData_assembled.count()
print(training_ratio)

0.8015591778880227


### Build a Decision Tree
Now that you've split the flights data into training and testing sets, you can use the training set to fit a Decision Tree model.

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

# Create a classifier object and fit to the training data
tree = DecisionTreeClassifier()
tree_model = tree.fit(combData_train)

# Create predictions for the testing data and take a look at the predictions
prediction = tree_model.transform(combData_test)
prediction.select('label', 'prediction', 'probability').show(5, False)

+-----+----------+---------------------------------------+
|label|prediction|probability                            |
+-----+----------+---------------------------------------+
|0    |0.0       |[0.8741258741258742,0.1258741258741259]|
|0    |0.0       |[0.8741258741258742,0.1258741258741259]|
|0    |0.0       |[0.8741258741258742,0.1258741258741259]|
|0    |0.0       |[0.8741258741258742,0.1258741258741259]|
|0    |0.0       |[0.8741258741258742,0.1258741258741259]|
+-----+----------+---------------------------------------+
only showing top 5 rows



### Evaluate the Decision Tree
You can assess the quality of your model by evaluating how well it performs on the testing data. Because the model was not trained on these data, this represents an objective assessment of the model.

A confusion matrix gives a useful breakdown of predictions versus known values. It has four cells which represent the counts of:

- True Negatives (TN) — model predicts negative outcome & known outcome is negative
- True Positives (TP) — model predicts positive outcome & known outcome is positive
- False Negatives (FN) — model predicts negative outcome but known outcome is positive
- False Positives (FP) — model predicts positive outcome but known outcome is negative.

In [ ]:
# Create a confusion matrix
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label = 1').count()
FP = prediction.filter('prediction = 1 AND label = 0').count()

# Accuracy measures the proportion of correct predictions
accuracy = (TN + TP) / (TN + TP + FN + FP)
print(accuracy)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|   19|
|    0|       0.0|  144|
|    1|       1.0|   93|
|    0|       1.0|   24|
+-----+----------+-----+

0.8464285714285714


## Logistic Regression


### Build a Logistic Regression model
You've already built a Decision Tree model using the flights data. Now you're going to create a Logistic Regression model on the same data.

The objective is to predict whether a flight is likely to be delayed by at least 15 minutes (label 1) or not (label 0).

Although you have a variety of predictors at your disposal, you'll only use the `mon`, `depart` and `duration` columns for the moment. These are numerical features which can immediately be used for a Logistic Regression model. You'll need to do a little more work before you can include categorical features.

In [ ]:
from pyspark.ml.classification import LogisticRegression

# Selecting numeric columns
combData_train_num = combData_train.select("teamLevel", 'count_gameclicks', 'count_hits', 'features', 'label')
combData_test_num = combData_test.select("teamLevel", "count_gameclicks", "count_hits", 'features', 'label')

# Create classifier object and train on training data
logistic = LogisticRegression().fit(combData_train_num)

# Create a predictions for the test data and show confusion matrix
prediction = logistic.transform(combData_test_num)
prediction.groupBy("label", "prediction").count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|   18|
|    0|       0.0|  143|
|    1|       1.0|   94|
|    0|       1.0|   25|
+-----+----------+-----+



### Evaluate the Logistic Regression model
Accuracy is generally not a very reliable metric because it can be biased by the most common target class.

There are two other useful metrics:

- precision and
- recall.

Precision is the proportion of positive predictions which are correct. For all flights which are predicted to be delayed, what proportion is actually delayed?

Recall is the proportion of positives outcomes which are correctly predicted. For all delayed flights, what proportion is correctly predicted by the model?

The precision and recall are generally formulated in terms of the positive target class. But it's also possible to calculate weighted versions of these metrics which look at both target classes.

In [ ]:
# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label = 1').count()
FP = prediction.filter('prediction = 1 AND label = 0').count()

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

# Calculate precision and recall
precision = TP / (TP + FP)
recall = TP / (TP + FN)
print('precision = {:.2f}\nrecall   = {:.2f}'.format(precision, recall))

# Find weighted precision
multi_evaluator = MulticlassClassificationEvaluator()
weighted_precision = multi_evaluator.evaluate(prediction, {multi_evaluator.metricName: "weightedPrecision"})

# Find AUC
binary_evaluator = BinaryClassificationEvaluator()
auc = binary_evaluator.evaluate(prediction, {binary_evaluator.metricName: "areaUnderROC"})

precision = 0.79
recall   = 0.84
